In [13]:
import os
import cv2
import dlib
import numpy as np
import inception_resnet_v1
import tensorflow as tf
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import matplotlib.pyplot as plt

In [14]:
tf.reset_default_graph()

sess = tf.Session()
images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)

train_mode = tf.placeholder(tf.bool)

age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm,
                                                            keep_probability=0.8,
                                                            phase_train=train_mode,
                                                            weight_decay=1e-5)

# Predict gender and age
gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)

init_op = tf.group(tf.global_variables_initializer(),
                  tf.local_variables_initializer())

sess.run(init_op)
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state("../gender_age_tf/models/")
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("restore model!")
else:
    pass 

INFO:tensorflow:Restoring parameters from ../gender_age_tf/models/model.ckpt-14001
restore model!


In [15]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=160)

In [16]:
img_size = 160

In [17]:
image_path = "/home/damvantai/Desktop/ss/"

In [33]:
img = cv2.imread(image_path + "Ba1MxF8A6sF.1.jpg")

In [34]:
image_path + "Ba1MxF8A6sF.1.jpg"

'/home/damvantai/Desktop/nguoidan/Ba1MxF8A6sF.1.jpg'

In [41]:
img = cv2.imread(image_path + "Ba1MxF8A6sF.1.jpg")

In [78]:
img = cv2.imread("/home/damvantai/Desktop/faces/1.jpg")

In [79]:
img.shape

(130, 129, 3)

In [80]:
detected = detector(img, 1)

In [81]:
detected

rectangles[]

In [35]:
def draw_label(image, point, label, font=cv2.FONT_HERSHEY_SIMPLEX,
               font_scale=1, thickness=2):
    size = cv2.getTextSize(label, font, font_scale, thickness)[0]
    x, y = point
    cv2.rectangle(image, (x, y - size[1]), (x + size[0], y), (255, 0, 0), cv2.FILLED)
    cv2.putText(image, label, point, font, font_scale, (255, 255, 255), thickness)


In [77]:
num_girl = 1
for i in os.listdir(image_path):
#     print(i)
    img = cv2.imread(image_path + i)
#     print(img.shape)
    input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_h, img_w, _ = np.shape(input_img)
    
    
    detected = detector(input_img, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    for i, d in enumerate(detected):
        faces[i, :, :, :] = fa.align(input_img, gray, detected[i])
        
    if len(detected) > 0:
        ages, genders = sess.run([age, gender], feed_dict={images_pl: faces, train_mode: False})
    
    
    for i, d in enumerate(detected):
        label = "{}, {}".format(int(ages[i]), "F" if genders[i] == 0 else "M")
#         draw_label(img, (d.left(), d.top()), label)
        if (genders[i]) == 0:
#           
            face_girl = img[d.top()-20:d.bottom()+20, d.left()-20:d.right()+20]
            
            cv2.imwrite("/home/damvantai/Desktop/faces/{}.jpg".format(num_girl),face_girl)
            num_girl += 1
#     plt.imshow(img)

## Detect girl from multi folder

In [1]:
import os
import cv2
import dlib
import numpy as np
import inception_resnet_v1
import tensorflow as tf
from imutils.face_utils import FaceAligner
from imutils.face_utils import rect_to_bb
import matplotlib.pyplot as plt

Instructions for updating:
Use the retry module or similar alternatives.


In [2]:
tf.reset_default_graph()

sess = tf.Session()
images_pl = tf.placeholder(tf.float32, shape=[None, 160, 160, 3], name='input_image')
images_norm = tf.map_fn(lambda frame: tf.image.per_image_standardization(frame), images_pl)

train_mode = tf.placeholder(tf.bool)

age_logits, gender_logits, _ = inception_resnet_v1.inference(images_norm,
                                                            keep_probability=0.8,
                                                            phase_train=train_mode,
                                                            weight_decay=1e-5)

# Predict gender and age
gender = tf.argmax(tf.nn.softmax(gender_logits), 1)
age_ = tf.cast(tf.constant([i for i in range(0, 101)]), tf.float32)
age = tf.reduce_sum(tf.multiply(tf.nn.softmax(age_logits), age_), axis=1)

init_op = tf.group(tf.global_variables_initializer(),
                  tf.local_variables_initializer())

sess.run(init_op)
saver = tf.train.Saver()
ckpt = tf.train.get_checkpoint_state("../gender_age_tf/models/")
if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)
    print("restore model!")
else:
    pass 

INFO:tensorflow:Restoring parameters from ../gender_age_tf/models/model.ckpt-14001
restore model!


In [3]:
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
fa = FaceAligner(predictor, desiredFaceWidth=160)

In [4]:
img_size = 160

In [5]:
image_path = "/home/damvantai/Desktop/faces/"

In [7]:
for i in os.listdir(image_path):
    num_girl = 1
    image_path_child = image_path + i + "/"
    print(image_path_child)
    for j in os.listdir(image_path_child):
        img = cv2.imread(image_path_child + j)
        input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        detected = detector(input_img, 1)
        faces = np.empty((len(detected), img_size, img_size, 3))
        
        for index, d in enumerate(detected):
            faces[index, :, :, :] = fa.align(input_img, gray, detected[index])
        
        if len(detected) == 1:
            ages, genders = sess.run([age, gender], feed_dict={images_pl: faces, train_mode: False})
            if genders[0] == 0:
                continue
            else:
                os.remove(image_path_child + j)
        else:
            os.remove(image_path_child + j)

/home/damvantai/Desktop/faces/1/
/home/damvantai/Desktop/faces/2/
/home/damvantai/Desktop/faces/3/


In [ ]:
num_girl = 1
for i in os.listdir(image_path):
#     print(i)
    img = cv2.imread(image_path + i)
#     print(img.shape)
    input_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
#     img_h, img_w, _ = np.shape(input_img)
    
    
    detected = detector(input_img, 1)
    faces = np.empty((len(detected), img_size, img_size, 3))
    
    for i, d in enumerate(detected):
        faces[i, :, :, :] = fa.align(input_img, gray, detected[i])
        
    if len(detected) > 0:
        ages, genders = sess.run([age, gender], feed_dict={images_pl: faces, train_mode: False})
        
    
    for i, d in enumerate(detected):
        label = "{}, {}".format(int(ages[i]), "F" if genders[i] == 0 else "M")
#         draw_label(img, (d.left(), d.top()), label)
        if (genders[i]) == 0:
#           
            face_girl = img[d.top()-20:d.bottom()+20, d.left()-20:d.right()+20]
            
            cv2.imwrite("/home/damvantai/Desktop/faces/{}.jpg".format(num_girl),face_girl)
            num_girl += 1
#     plt.imshow(img)